In [ ]:
'''
CREATE TABLE ko_pitcherbox (
    gameid INT,
    hw VARCHAR(20),
    abbreviation VARCHAR(5),
    pitcherName VARCHAR(30),
    pitcherid INT,
    IP DECIMAL(3,1),
    H INT,
    R INT,
    ER INT,
    BB INT,
    K INT,
    HR INT,
    ERA DECIMAL(5,2)
    note VARCHAR(30)
)
'''

In [3]:
import statsapi
sched = statsapi.schedule(start_date='01/01/2024',end_date='12/30/2024')
# team id key dict
t_info = {}

from team import team
t = team()
t.getAll()
for tt in t.data:
    t_info[tt["teamid"]] = tt['teamKey']
    
game_list = []
for i in sched:
    if i['status'] == 'Final' and i['game_type'] == "R":
        game = {}
        game['gameid'] = i['game_id']
        game['date'] = i['game_date']
        game['awayTeam'] = i['away_name']
        game['homeTeam'] = i['home_name']
        game['awayKey'] = t_info[i['away_id']]
        game['homeKey'] = t_info[i['home_id']]
        game['awayid'] = i['away_id']
        game['homeid'] = i['home_id']
        game['homeScore'] = i['home_score']
        game['awayScore'] = i['away_score']
        game_list.append(game)

In [6]:
import statsapi
from pitcherbox import pitcherbox
b = pitcherbox()
b.truncate()
count = 0
sql = f"INSERT INTO `{b.tn}` ("
for field in b.fields:
    sql += f"`{field}`,"
    count +=1
sql = sql[0:-1] + ') VALUES ('
tokens = ("%s," * count)[0:-1]
sql += tokens + ');'

for g in game_list:
    df = statsapi.boxscore_data(g['gameid'], timecode=None)
    p_info = []
    n=0
    for k in df['awayPitchers']:
        if n > 0:
            p = {}
            p['gameid'] = g['gameid']
            p['hw'] = 'away'
            p['teamid'] = df['teamInfo']['away']['id']
            p['abbreviation'] = df['teamInfo']['away']['abbreviation']
            p['pitcherName'] = k['name']
            p['pitcherid'] = k['personId']
            p['IP'] = k['ip']
            p['H'] = k['h']
            p['R'] = k['r']
            p['ER'] = k['er']
            p['BB'] = k['bb']
            p['K'] = k['k']
            p['HR'] = k['hr']
            try:
                p['ERA'] = float(k['era'])
            except:
                p['ERA'] = 999
            p['note'] = k['note']
            p_info.append(p)
        n += 1
    away_total = {}
    away_total['gameid'] = g['gameid']
    away_total['hw'] = 'away'
    away_total['teamid'] = df['teamInfo']['away']['id']
    away_total['abbreviation'] = df['teamInfo']['away']['abbreviation']
    away_total['pitcherName'] = 'Totals'
    away_total['pitcherid'] = 0
    away_total['IP'] = df['awayPitchingTotals']['ip']
    away_total['R'] = df['awayPitchingTotals']['r']
    away_total['H'] = df['awayPitchingTotals']['h']
    away_total['ER'] = df['awayPitchingTotals']['er']
    away_total['BB'] = df['awayPitchingTotals']['bb']
    away_total['K'] = df['awayPitchingTotals']['k']
    away_total['HR'] = df['awayPitchingTotals']['hr']
    away_total['ERA'] = 0
    away_total['note'] = ""
    p_info.append(away_total)
    n=0
    for k in df['homePitchers']:
        if n > 0:
            p = {}
            p['gameid'] = g['gameid']
            p['hw'] = 'home'
            p['teamid'] = df['teamInfo']['home']['id']
            p['abbreviation'] = df['teamInfo']['home']['abbreviation']
            p['pitcherName'] = k['name']
            p['pitcherid'] = k['personId']
            p['IP'] = k['ip']
            p['H'] = k['h']
            p['R'] = k['r']
            p['ER'] = k['er']
            p['BB'] = k['bb']
            p['K'] = k['k']
            p['HR'] = k['hr']
            try:
                p['ERA'] = float(k['era'])
            except:
                p['ERA'] = 999
            p['note'] = k['note']
            p_info.append(p)
        n += 1
    home_total = {}
    home_total['gameid'] = g['gameid']
    home_total['hw'] = 'home'
    home_total['teamid'] = df['teamInfo']['home']['id']
    home_total['abbreviation'] = df['teamInfo']['home']['abbreviation']
    home_total['pitcherName'] = 'Totals'
    home_total['pitcherid'] = 0
    home_total['IP'] = df['homePitchingTotals']['ip']
    home_total['R'] = df['homePitchingTotals']['r']
    home_total['H'] = df['homePitchingTotals']['h']
    home_total['ER'] = df['homePitchingTotals']['er']
    home_total['BB'] = df['homePitchingTotals']['bb']
    home_total['K'] = df['homePitchingTotals']['k']
    home_total['HR'] = df['homePitchingTotals']['hr']
    home_total['ERA'] = 0
    home_total['note'] = ""
    p_info.append(home_total)

    vals = []
    for p in p_info:
        val = []
        for field in b.fields:
            val.append(p[field])
        vals.append(val) 

    b.cur.executemany(sql,vals)

In [2]:
from pitcherbox import pitcherbox
b = pitcherbox()
b.truncate()
count = 0
sql = f"INSERT INTO `{b.tn}` ("
for field in b.fields:
    sql += f"`{field}`,"
    count +=1
sql = sql[0:-1] + ') VALUES ('
tokens = ("%s," * count)[0:-1]
sql += tokens + ');'


vals = []
for p in p_info:
    val = []
    for field in b.fields:
        val.append(p[field])
    vals.append(val) 

        

print(len(p_info))
b.cur.executemany(sql,vals)

10


10

In [7]:
b.getByMultiField(["gameid",'hw'], [745373,'away'])